Train a Auto-Encoder to reduce the dimensionality of the features. Then evaluate the effectiveness via training a linear SVM with the new features.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model

Using TensorFlow backend.


Load data and split it into training set (60%) and test set (40%)

In [2]:
features_file = 'AwA2-features/Animals_with_Attributes2/Features/ResNet101/AwA2-features.txt'
labels_file = 'AwA2-features/Animals_with_Attributes2/Features/ResNet101/AwA2-labels.txt'

# There is in total 37322 images of 50 classes. Each image is represented as a 2048 dimensional feature
features = np.loadtxt(features_file) # shape (37322, 2048)
labels = np.loadtxt(labels_file) # shape (37322, )

# Split each and all classes into training set (60%) and test set (40%)
# set random_state to an int for reproducibility
X_train, X_test, Y_train, Y_test = train_test_split(
    features, labels, train_size=0.6, test_size=0.4, random_state=0, stratify=labels)


Design an auto-encoder architecture (with only fully connected layers) and train the neural network in the conventional supervised learning manner. The labels of the training set is, however, no longer the class categories but rather the input itself.

In [3]:
d = X_train.shape[1]

# Define the encoder dimension
encoding_dim = 250

layer0 = Input(shape=(d, ))

# Encoder Layers
encoding_layer1 = Dense(2000, activation='relu')(layer0)
encoding_layer2 = Dense(1750, activation='relu')(encoding_layer1)
encoding_layer3 = Dense(1500, activation='relu')(encoding_layer2)
encoding_layer4 = Dense(1250, activation='relu')(encoding_layer3)
encoding_layer5 = Dense(1000, activation='relu')(encoding_layer4)
encoding_layer6 = Dense(750, activation='relu')(encoding_layer5)
encoding_layer7 = Dense(500, activation='relu')(encoding_layer6)
encoding_layer8 = Dense(encoding_dim, activation='relu')(encoding_layer7)

# Decoder layers
decoding_layer1 = Dense(500, activation='relu')(encoding_layer8)
decoding_layer2 = Dense(750, activation='relu')(decoding_layer1)
decoding_layer3 = Dense(1000, activation='relu')(decoding_layer2)
decoding_layer4 = Dense(1250, activation='relu')(decoding_layer3)
decoding_layer5 = Dense(1500, activation='relu')(decoding_layer4)
decoding_layer6 = Dense(1750, activation='relu')(decoding_layer5)
decoding_layer7 = Dense(2000, activation='relu')(decoding_layer6)
decoding_layer8 = Dense(d, activation='relu')(decoding_layer7)

In [4]:
# Define models
autoencoder = Model(inputs=layer0, outputs=decoding_layer8)
encoder = Model(inputs=layer0, outputs=encoding_layer8)

# Compile the Model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train, X_train, batch_size=64, epochs=60, validation_split=0.1, shuffle=False)

# Save the model
encoder.save('encoder.h5')

autoencoder.summary()

encoder.summary()

Train on 20153 samples, validate on 2240 samples
Epoch 1/60
20153/20153 [==============================] - 1834s 91ms/step - loss: 0.5377 - val_loss: 0.5025
Epoch 2/60
20153/20153 [==============================] - 105s 5ms/step - loss: 0.4472 - val_loss: 0.4064
Epoch 3/60
20153/20153 [==============================] - 104s 5ms/step - loss: 0.3955 - val_loss: 0.3861
Epoch 4/60
20153/20153 [==============================] - 102s 5ms/step - loss: 0.3800 - val_loss: 0.3701
Epoch 5/60
20153/20153 [==============================] - 101s 5ms/step - loss: 0.3689 - val_loss: 0.3760
Epoch 6/60
20153/20153 [==============================] - 99s 5ms/step - loss: 0.3725 - val_loss: 0.3853
Epoch 7/60
20153/20153 [==============================] - 100s 5ms/step - loss: 0.3600 - val_loss: 0.3526
Epoch 8/60
20153/20153 [==============================] - 101s 5ms/step - loss: 0.3420 - val_loss: 0.3374
Epoch 9/60
20153/20153 [==============================] - 105s 5ms/step - loss: 0.3339 - val_loss: 0.3

Now use the encoder part to reduce dimensionality of features

In [5]:
# Load the model
encoder = load_model('encoder.h5')

# Reduce the dimensionality by passing all the features to the encoder part
reduced_X_train = encoder.predict(X_train)
reduced_X_test = encoder.predict(X_test)

# Sanity Check: Make sure the dimensionality of the reduced features is 250
assert reduced_X_train.shape[0] == X_train.shape[0]
assert reduced_X_train.shape[1] == encoding_dim
assert reduced_X_test.shape[0] == X_test.shape[0]
assert reduced_X_test.shape[1] == encoding_dim

/home/xiang/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Now let's try training a linear SVM with the new features!

In [6]:
clf = SVC(kernel='linear', C=0.001)
clf.fit(reduced_X_train, Y_train)
accuracy = clf.score(reduced_X_test, Y_test)

print('Accuracy: %f' % (accuracy))

Accuracy: 0.803470
